In [13]:
# pip install google-cloud-pubsub

# pip install apache_beam

import argparse
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window

serviceAccount = 'key.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/gp-ct-sbox-tax-globalbts/subscriptions/movies-sub'

parser = argparse.ArgumentParser()
known_args, pipeline_args = parser.parse_known_args(None)

options= PipelineOptions(pipeline_args)
options.view_as(StandardOptions).streaming= True
p = beam.Pipeline(options=options)

comedy_movies = 'projects/gp-ct-sbox-tax-globalbts/topics/comedy_movies'

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Filter movies with comedy genre' >> beam.Filter(lambda row: ("Comedy" in row[2]))
    | 'Converting to byte String' >> beam.Map(lambda row: (''.join(row).encode('utf-8')) )
    | 'Publish to output topic' >> beam.io.WriteToPubSub(comedy_movies)
)

result = p.run()
result.wait_until_finish()

usage: ipykernel_launcher.py [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                             [--project PROJECT] [--job_name JOB_NAME]
                             [--staging_location STAGING_LOCATION]
                             [--temp_location TEMP_LOCATION] [--region REGION]
                             [--service_account_email SERVICE_ACCOUNT_EMAIL]
                             [--no_auth]
                             [--template_location TEMPLATE_LOCATION]
                             [--label LABELS] [--update]
                             [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                             [--enable_streaming_engine]
                             [--dataflow_kms_key DATAFLOW_KMS_KEY]
                             [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                             [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                             [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
ipykernel_launcher.py: erro

Traceback (most recent call last):
  File "c:\python395\lib\argparse.py", line 1851, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "c:\python395\lib\argparse.py", line 2060, in _parse_known_args
    start_index = consume_optional(start_index)
  File "c:\python395\lib\argparse.py", line 2000, in consume_optional
    take_action(action, args, option_string)
  File "c:\python395\lib\argparse.py", line 1912, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "c:\python395\lib\argparse.py", line 2444, in _get_values
    self._check_value(action, value)
  File "c:\python395\lib\argparse.py", line 2500, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --flexrs_goal: invalid choice: 'C:\\Users\\ABHISH~1.KUM\\AppData\\Local\\Temp\\tmp-34632zHSuQHCjlCDs.json' (choose from 'COST_OPTIMIZED', 'SPEED_OPTIMIZED')

During handling of the above exception, another exception occurred

TypeError: object of type 'NoneType' has no len()